In [51]:
from rich import print as rprint
import torch
import torch.nn as nn
import torch.nn.functional as F

In [59]:
with open('./input.txt', encoding='utf-8') as f:
    text = f.read()

rprint(f'Text length: {len(text)}')

Text length: 1115394

In [61]:
rprint(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

In [4]:
# Unique characters in text
chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)

rprint(f'Vocabulary size: {VOCAB_SIZE}')
rprint(f'{"".join(chars)}')

Vocabulary size: 65

!$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz

## Defining embeddings

In [5]:
# Character mapping
s_to_i = {s: i for i, s in enumerate(chars)}
i_to_s = {i: s for i, s in enumerate(chars)}

encode = lambda s: [s_to_i[c] for c in s]
decode = lambda l: ''.join([i_to_s[i] for i in l])

rprint(f'Encoded: {encode("Hello World")}')

rprint(f'Decoded: {decode([56, 50, 42])}')

Encoded: [20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]

Decoded: rld

In [6]:
# Encoding the input
data = torch.tensor(encode(text), dtype=torch.long)
rprint(f'Encoded data shape: {data.shape}')
rprint(f'Encoded data: {data[:100]}')

Encoded data shape: torch.Size([1115394])

Encoded data: tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [7]:
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

## Generating batches

In [8]:
BLOCK_SIZE = 8

X = train_data[:BLOCK_SIZE]
y = train_data[1:BLOCK_SIZE + 1]

for t in range(BLOCK_SIZE):
    rprint(f'Input: {X[:t+1]} , Output:{y[t]}')

Input: tensor([18]) , Output:47

Input: tensor([18, 47]) , Output:56

Input: tensor([18, 47, 56]) , Output:57

Input: tensor([18, 47, 56, 57]) , Output:58

Input: tensor([18, 47, 56, 57, 58]) , Output:1

Input: tensor([18, 47, 56, 57, 58,  1]) , Output:15

Input: tensor([18, 47, 56, 57, 58,  1, 15]) , Output:47

Input: tensor([18, 47, 56, 57, 58,  1, 15, 47]) , Output:58

In [9]:
torch.manual_seed(1337)

BATCH_SIZE = 4
BLOCK_SIZE = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    X = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    return X, y

xb, yb = get_batch('train')
rprint(f'Inputs')
rprint(f'X shape: {xb.shape}')
rprint(f'X: {xb}')
rprint(f'Outputs')
rprint(f'y shape: {yb.shape}')
rprint(f'y: {yb}')

rprint('-' * 50)

Inputs

X shape: torch.Size([4, 8])

X: tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

Outputs

y shape: torch.Size([4, 8])

y: tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

--------------------------------------------------

In [10]:
for b in range(BATCH_SIZE):
    for t in range(BLOCK_SIZE):
        context = xb[b, :t+1]
        target = yb[b, t]
        rprint(f'Context: {context} , Target: {target}')
        rprint(f'Context: {decode(context.tolist())} , Target: {decode([int(target)])}')
    rprint('-' * 50)

Context: tensor([24]) , Target: 43

Context: L , Target: e

Context: tensor([24, 43]) , Target: 58

Context: Le , Target: t

Context: tensor([24, 43, 58]) , Target: 5

...

Context: tensor([25, 17, 27, 10,  0, 21,  1]) , Target: 54

Context: MEO:
I  , Target: p

Context: tensor([25, 17, 27, 10,  0, 21,  1, 54]) , Target: 39

Context: MEO:
I p , Target: a

--------------------------------------------------

In [11]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


## Bigram Language Model

In [12]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            return logits, None
        
        B, T, C = logits.shape
        logits = logits.view(B * T, C)
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)

            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_next], dim=1)
            
        return idx

model = BigramLanguageModel(VOCAB_SIZE)

out = model(xb, yb)
rprint(f'Output shape: {out[0].shape}')
rprint(f'Loss: {out[1]}')
rprint(f'Generate: {model.generate(xb, 5)}')

Output shape: torch.Size([32, 65])

Loss: 4.878634929656982

Generate: tensor([[24, 43, 58,  5, 57,  1, 46, 43, 60, 60, 12, 55, 28],
        [44, 53, 56,  1, 58, 46, 39, 58, 31,  0, 60, 60, 47],
        [52, 58,  1, 58, 46, 39, 58,  1,  7, 23, 20, 29, 16],
        [25, 17, 27, 10,  0, 21,  1, 54,  4, 29, 10, 17, 22]])

In [13]:
rprint(decode(model.generate(torch.zeros((1,1), dtype=torch.long), 100)[0].tolist()))

-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3!dcbf

## Train the model

In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0.01
)

In [15]:
BATCH_SIZE = 32

for steps in range(10000):
    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if steps % 1000 == 0:
        rprint(f'Step: {steps} , Loss: {loss}')

rprint(f'Loss: {loss.item()}')

Step: 0 , Loss: 4.713774681091309

Step: 1000 , Loss: 3.7239983081817627

Step: 2000 , Loss: 3.13909649848938

Step: 3000 , Loss: 2.6936428546905518

Step: 4000 , Loss: 2.602484703063965

...

Step: 6000 , Loss: 2.5261800289154053

Step: 7000 , Loss: 2.4512107372283936

Step: 8000 , Loss: 2.4537270069122314

Step: 9000 , Loss: 2.508976936340332

Loss: 2.52470326423645

In [16]:
rprint(decode(model.generate(torch.zeros((1,1), dtype=torch.long), 300)[0].tolist()))

ng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

## Building up to self attention

In [17]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [18]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, dim=0)

In [19]:
rprint(f'X[0]: {x[0]}')
rprint(f'Xbow[0]: {xbow[0]}')

X[0]: tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

Xbow[0]: tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

Notice how the first row is the same, but every subsequent $i^{th}$ row is a simple average of the 0 to $i^{th}$ rows.

### Matrix multiplication trick

In [20]:
# Matrix multiplication
torch.manual_seed(42)
a = torch.ones(4,4)
b = torch.randint(0, 10, (4,3)).float()
c = a @ b

rprint(f'a: {a}')
rprint(f'b: {b}')
rprint(f'c: {c}')

a: tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

b: tensor([[2., 7., 6.],
        [4., 6., 5.],
        [0., 4., 0.],
        [3., 8., 4.]])

c: tensor([[ 9., 25., 15.],
        [ 9., 25., 15.],
        [ 9., 25., 15.],
        [ 9., 25., 15.]])

In [21]:
# Matrix multiplication to sum previous rows. This is using a lower triangular matrix as a mask
a_ = torch.tril(torch.ones((4,4)))
rprint(f'a_: {a_}')
rprint(f'b: {b}')
rprint(f'a_ @ b: {a_ @ b}')

a_: tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])

b: tensor([[2., 7., 6.],
        [4., 6., 5.],
        [0., 4., 0.],
        [3., 8., 4.]])

a_ @ b: tensor([[ 2.,  7.,  6.],
        [ 6., 13., 11.],
        [ 6., 17., 11.],
        [ 9., 25., 15.]])

In [22]:
# Taking the average as a normalization step
a_normalized = a_ / torch.sum(a_, dim=1, keepdim=True)
rprint(f'a_normalized: {a_normalized}')

rprint(f'a_normalized @ b: {a_normalized @ b}')

a_normalized: tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500]])

a_normalized @ b: tensor([[2.0000, 7.0000, 6.0000],
        [3.0000, 6.5000, 5.5000],
        [2.0000, 5.6667, 3.6667],
        [2.2500, 6.2500, 3.7500]])

### Using matrix multiplication to make it efficient

In [23]:
weights = torch.tril(torch.ones(T, T))
weights = weights / torch.sum(weights, dim=1, keepdim=True)
rprint(f'Weights: {weights}')

Weights: tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [24]:
xbow_matrix = weights @ x
rprint(f'Xbow matrix: {xbow_matrix}')

Xbow matrix: tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

### Using softmax


In [25]:
tril = torch.tril(torch.ones((T, T)))
weights = torch.zeros((T, T))

weights = weights.masked_fill(tril == 0, float('-inf'))

weights = F.softmax(weights, dim=-1)

xbow_softmax = weights @ x
rprint(f'Xbow softmax: {xbow_softmax}')

torch.allclose(xbow_matrix, xbow_softmax)

Xbow softmax: tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

True

## Self-attention

In [29]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

tril = torch.tril(torch.ones(T, T))
weights = torch.randn((T, T))
weights = weights.masked_fill(tril == 0, float('-inf'))
weights = F.softmax(weights, dim=-1)

out = weights @ x


### Single head of self attention

In [45]:
HEAD_SIZE = 16
query = nn.Linear(C, HEAD_SIZE, bias=False)
key = nn.Linear(C, HEAD_SIZE, bias=False)
value = nn.Linear(C, HEAD_SIZE, bias=False)

rprint(f'Query: {query}')
rprint(f'Key: {key}')
rprint(f'Value: {value}')

Query: Linear(in_features=32, out_features=16, bias=False)

Key: Linear(in_features=32, out_features=16, bias=False)

Value: Linear(in_features=32, out_features=16, bias=False)

In [46]:
q = query(x)
k = key(x)
v = value(x)

rprint(f'Query shape: {q.shape}')
rprint(f'Key shape: {k.shape}')
rprint(f'Value shape: {v.shape}')

Query shape: torch.Size([4, 8, 16])

Key shape: torch.Size([4, 8, 16])

Value shape: torch.Size([4, 8, 16])

The weight is calculated using:
$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

Reference: https://arxiv.org/pdf/1706.03762.pdf

In [47]:
tril = torch.tril(torch.ones(T, T))
weights = q @ k.transpose(-2, -1) ** (HEAD_SIZE ** -0.5) # scaled dot product attention
weights = weights.masked_fill(tril == 0, float("-inf"))
weights = F.softmax(weights, dim=-1)

rprint(f'Weights: {weights[0]}')

Weights: tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8304, 0.1696, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6049, 0.1018, 0.2933, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4319, 0.2394, 0.3199, 0.0088, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1226, 0.0152, 0.0291, 0.4244, 0.4086, 0.0000, 0.0000, 0.0000],
        [0.6042, 0.1339, 0.1163, 0.0263, 0.0698, 0.0495, 0.0000, 0.0000],
        [0.1153, 0.0733, 0.0450, 0.1496, 0.2315, 0.0444, 0.3408, 0.0000],
        [0.2276, 0.0767, 0.0598, 0.0366, 0.1111, 0.2532, 0.1576, 0.0774]],
       grad_fn=<SelectBackward0>)

In [50]:
out = weights @ v
rprint(f'Out: {out.shape}')

Out: torch.Size([4, 8, 16])